# Kolmogorov Complexity to Compress DNA Sequences Using Python
Long form blogpost can be found over here: [yacinemahdid.com](yacinemahdid.com)

The data used can be found over here: [Kaggle/Humans.txt file](https://www.kaggle.com/nageshsingh/dna-sequence-dataset/version/1?select=human.txt)

In [2]:
import pandas as pd
HUMAN_DNA_DATASET = "/home/yacine/Desktop/archive/human.txt"
df = pd.read_csv(HUMAN_DNA_DATASET, sep = "\t", names = ['sequence','class'], skiprows=1)

# No idea what the class variable is though as it wasn't mentioned in the dataset on Kaggle.
df

,sequence,class
0,ATGCCCCAACTAAATACTACCGTATGGCCCACCATAATTACCCCCA...,4
1,ATGAACGAAAATCTGTTCGCTTCATTCATTGCCCCCACAATCCTAG...,4
2,ATGTGTGGCATTTGGGCGCTGTTTGGCAGTGATGATTGCCTTTCTG...,3
3,ATGTGTGGCATTTGGGCGCTGTTTGGCAGTGATGATTGCCTTTCTG...,3
4,ATGCAACAGCATTTTGAATTTGAATACCAGACCAAAGTGGATGGTG...,3
...,...,...
4375,ATGGAAGATTTGGAGGAAACATTATTTGAAGAATTTGAAAACTATT...,0
4376,ATGCAGTCCTTTCGGGAGCAAAGCAGTTACCACGGAAACCAGCAAA...,6
4377,ATGCAGTCCTTTCGGGAGCAAAGCAGTTACCACGGAAACCAGCAAA...,6
4378,ATGGGGCACCTGGTTTGCTGTCTGTGTGGCAAGTGGGCCAGTTACC...,6


In [66]:
# Source is right here: https://github.com/Naereen/Lempel-Ziv_Complexity/blob/master/src/lempel_ziv_complexity.py

def lempel_ziv_complexity(sequence):
    r""" Manual implementation of the Lempel-Ziv complexity.
    It is defined as the number of different substrings encountered as the stream is viewed from begining to the end.
    As an example:
    >>> s = '1001111011000010'
    >>> lempel_ziv_complexity(s)  # 1 / 0 / 01 / 11 / 10 / 110 / 00 / 010
    8
    Marking in the different substrings the sequence complexity :math:`\mathrm{Lempel-Ziv}(s) = 8`: :math:`s = 1 / 0 / 01 / 11 / 10 / 110 / 00 / 010`.
    - See the page https://en.wikipedia.org/wiki/Lempel-Ziv_complexity for more details.
    Other examples:
    >>> lempel_ziv_complexity('1010101010101010')  # 1, 0, 10, 101, 01, 010, 1010
    7
    >>> lempel_ziv_complexity('1001111011000010000010')  # 1, 0, 01, 11, 10, 110, 00, 010, 000
    9
    >>> lempel_ziv_complexity('100111101100001000001010')  # 1, 0, 01, 11, 10, 110, 00, 010, 000, 0101
    10
    - Note: it is faster to give the sequence as a string of characters, like `'10001001'`, instead of a list or a numpy array.
    - Note: see this notebook for more details, comparison, benchmarks and experiments: https://Nbviewer.Jupyter.org/github/Naereen/Lempel-Ziv_Complexity/Short_study_of_the_Lempel-Ziv_complexity.ipynb
    - Note: there is also a Cython-powered version, for speedup, see :download:`lempel_ziv_complexity_cython.pyx`.
    """
    sub_strings = set()
    n = len(sequence)

    ind = 0
    inc = 1
    while True:
        if ind + inc > len(sequence):
            break
        sub_str = sequence[ind : ind + inc]
        if sub_str in sub_strings:
            inc += 1
        else:
            sub_strings.add(sub_str)
            ind += inc
            inc = 1
    return len(sub_strings)

def string_to_binary(string_sequence):
    return ''.join(format(ord(x), 'b') for x in string_sequence)

def generate_window(sequence, window_size=250, jump_size=50):
    start = 0
    end = window_size
    
    while end < len(sequence):
        yield sequence[start:end]
        
        start = start + jump_size
        end = end + jump_size

def generate_lz_sequence(sequence):
    lz_sequence = []
    for window in generate_window(sequence, window_size=2, jump_size=1):
        encoded_sequence = string_to_binary(window)
        lz = lempel_ziv_complexity(encoded_sequence)
        lz_sequence.append(lz)
    
    return lz_sequence
    
# TODO create a graph out of this using matplotlib
generate_lz_graph("ABABBABABABABAAB")

[6, 6, 6, 7, 6, 6, 6, 6, 6, 6, 6, 6, 6, 7]